# Zomato Exercise

DEL ROSARIO, JAVIER

References: 

https://www.askpython.com/python-modules/pandas/combine-csv-files-using-python
https://stackoverflow.com/questions/48590268/pandas-get-the-most-frequent-values-of-a-column/48590361
https://www.geeksforgeeks.org/python-lambda-anonymous-functions-filter-map-reduce/

Zomato is a restaurant aggregator website. The platform provides information, menus and user-reviews of restaurants as well as food delivery options from partner restaurants in select cities. You have been provided 3 datasets:

- zomato.csv contains restaurant information like average_cost_for_two, cuisines
- location.csv contains location of the restaurants like address, city, locality & etc
- ratings.csv contains aggregate rating for each restaurant

The different files are connected using the `id` columns which can be found in the different files. The id column is a unique identifier to represent a restaurant. You can use this to **merge** the different files.

In [6]:
%pip install pandas
import pandas as pd
import numpy as np


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl (11.5 MB)


1. Merge the zomato, location & ratings file into a signle dataframe and store it to the `merged_df` variable **(3pts)**

In [9]:
zomato_df = pd.read_csv('zomato.csv')
location_df = pd.read_csv('location.csv')
ratings_df = pd.read_csv('ratings.csv')

merged_df = zomato_df.merge(location_df, on='id', how='inner').merge(ratings_df, on='id', how='inner')

In [11]:
assert (6830, 35)==merged_df.shape

cols = ['id', 'address', 'city', 'city_id', 'country_id', 'latitude',
       'locality', 'locality_verbose', 'longitude', 'zipcode',
       'aggregate_rating', 'rating_color', 'rating_text', 'votes',
       'average_cost_for_two', 'book_url', 'cuisines', 'currency', 'deeplink',
       'events_url', 'featured_image', 'has_online_delivery',
       'has_table_booking', 'include_bogo_offers', 'is_book_form_web_view',
       'is_delivering_now', 'is_table_reservation_supported',
       'is_zomato_book_res', 'menu_url', 'mezzo_provider', 'name',
       'photos_url', 'price_range', 'thumb', 'url']

assert sorted(merged_df.columns) == sorted(cols)

2. Create a function that returns the average aggregate rating for each city. Expected returned value is a Series datatype which contains name of city as index and average aggregate rating as value. **(2pts)**

In [22]:
def exercise_2(merged_df):
    avg_rating = merged_df.groupby('city')['aggregate_rating'].mean()
    print(avg_rating)
    return avg_rating


In [23]:
res_series = exercise_2(merged_df)
assert True == np.isclose(3.302075, res_series[res_series.index == 'Pasay City'],
                          rtol=1e-05, atol=1e-08, equal_nan=False)[0]
assert True == np.isclose(1.890741, res_series[res_series.index == 'Valenzuela City'],
                          rtol=1e-05, atol=1e-08, equal_nan=False)[0]
assert True == np.isclose(1.558333, res_series[res_series.index == 'Malabon City'],
                          rtol=1e-05, atol=1e-08, equal_nan=False)[0]



city
Caloocan City       1.953509
Las Piñas City      2.855738
Makati City         3.250512
Malabon City        1.558333
Mandaluyong City    2.951919
Manila              2.842652
Marikina City       2.672512
Muntinlupa City     2.962682
Parañaque City      3.130100
Pasay City          3.302075
Pasig City          2.916086
Pateros City        1.740000
Quezon City         2.887946
San Juan City       3.098462
Tagaytay City       2.959259
Taguig City         3.234105
Valenzuela City     1.890741
Name: aggregate_rating, dtype: float64


3. Using the apply function of pandas convert the rating_text to a numeric equivalent. You are to introduce a new column in merged_df called `rating_numeric` which is based on the value from rating_text: **(3pts)**

Excellent = 5 <br>
Very Good = 4 <br>
Good = 3 <br>
Average = 2 <br>
Poor = 1 <br>
Not rated = 0 <br>

In [25]:
rating_map = {
    'Excellent': 5,
    'Very Good': 4,
    'Good': 3,
    'Average': 2,
    'Poor': 1,
    'Not rated': 0
}

rating_numeric = merged_df.apply(lambda x: rating_map[x.rating_text], axis=1)

merged_df = merged_df.assign(rating_numeric=rating_numeric)

In [26]:
assert 868 == merged_df[merged_df.rating_numeric == 0].shape[0]
assert True == ('rating_numeric' in merged_df.columns)

4. Create a function which filters the merged dataframe to only contain rows coming in from a specific city. Expected returned value is a dataframe **(2pts)**

In [33]:
def exercise_4(merged_df, city):
    filtered_df = merged_df[merged_df.city == city]
    print(filtered_df)
    return filtered_df

In [35]:

assert 'Makati City' == exercise_4(merged_df, 'Makati City').city.unique()[0]
assert (976, 36) == exercise_4(merged_df, 'Makati City').shape

assert 'Mandaluyong City' == exercise_4(merged_df, 'Mandaluyong City').city.unique()[0]
assert (443, 36) == exercise_4(merged_df, 'Mandaluyong City').shape

assert (0, 36) == exercise_4(merged_df, 'Quezon  City').shape

            id  average_cost_for_two book_url                   cuisines  \
0      6316125                  1000      NaN            Japanese, Ramen   
1     18409457                  1300      NaN  Korean, Grill, Korean BBQ   
2      6307689                  1600      NaN         American, Filipino   
7      6304287                  1200      NaN                   Japanese   
10     6304833                   700      NaN                   Filipino   
...        ...                   ...      ...                        ...   
6598  18303726                   700      NaN     Healthy Food, Japanese   
6753   6301694                  1200      NaN                   Filipino   
6758  18351649                  1100      NaN                Finger Food   
6815   6317082                   500      NaN                 Vietnamese   
6817  18340454                   500      NaN                 Vietnamese   

     currency                      deeplink  \
0           P   zomato://restaurant/6316

5. Create a function to filter the dataframe to only contain certain columns passed in the function. Expected returned value is a dataframe **(1pt)**

In [42]:
def exercise_5(merged_df, cols_to_filter):
    filtered_df = merged_df[cols_to_filter]
    print(filtered_df)
    return filtered_df

In [44]:
cols = ['name', 'cuisines', 'city', 'aggregate_rating']
assert sorted(cols) == sorted(exercise_5(merged_df, cols).columns)
assert (6830, 4) == exercise_5(merged_df, cols).shape

cols = ['average_cost_for_two', 'book_url', 'cuisines', 'currency', 'deeplink',
       'events_url', 'featured_image', 'has_online_delivery',
       'has_table_booking', 'include_bogo_offers', 'is_book_form_web_view']
assert sorted(cols) == sorted(exercise_5(merged_df, cols).columns)
assert (6830, 11) == exercise_5(merged_df, cols).shape


                           name                                    cuisines  \
0             Mendokoro Ramenba                             Japanese, Ramen   
1            Soban K-Town Grill                   Korean, Grill, Korean BBQ   
2               Filling Station                          American, Filipino   
3                Romantic Baboy                                      Korean   
4                NIU by Vikings  Seafood, American, Mediterranean, Japanese   
...                         ...                                         ...   
6825              Trader Juan's                              Deli, Filipino   
6826                   Jollibee                                   Fast Food   
6827                 Goldilocks                            Bakery, Filipino   
6828  Red Buffalo Wings & Pizza                           American, Italian   
6829                  Greenwich                            Pizza, Fast Food   

                 city  aggregate_rating  
0        

6.  What is the most common restaurant name in the dataset? **(2pts)** Write down your answer and show code used to come up with the answer.

Starbucks

In [79]:
merged_df.name.mode()[0]

'Starbucks'

7. Which restaurant has the most expensive average cost for two? **(2pts)** Write down your answer and show code used to come up with the answer.

The Tasting Room

In [67]:
merged_df.loc[merged_df['average_cost_for_two'].idxmax(), 'name']

'The Tasting Room'

8. How many restaurants do not have a zipcode? **(1pt)** Write down your answer and show code used to come up with the answer.

6713

In [76]:

nozip = len(merged_df) - merged_df['zipcode'].count()
print(nozip)

6713


9. How many restaurants in the dataset allow online delivery? **(1pt)** Write down your answer and show code used to come up with the answer.

0

In [84]:
hasonlinedel = merged_df['has_online_delivery'].sum()
print(hasonlinedel)

0


10. Which locatlity has more expensive food? Poblacion or Salcedo Village? **(3pts)** Write down your answer and show code used to come up with the answer.


Salcedo

In [86]:
poblacion = merged_df[merged_df['locality'] == 'Poblacion']
salcedo = merged_df[merged_df['locality'] == 'Salcedo Village']

pob_cost = poblacion['average_cost_for_two'].mean()
sal_cost = salcedo['average_cost_for_two'].mean()

print(f'Poblacion food cost: {pob_cost}')
print(f'Salcedo food cost: {sal_cost}')

Poblacion food cost: 898.2142857142857
Salcedo food cost: 989.1025641025641
